<h1 align="center"><b>Search Algorithms</b></h1>
<h5 align="center">Uninformed and Informed search</h5>

---

### Table of Contents

1. [Preliminary Steps](#1-preliminary-steps)
2. [Uninformed Search](#2-uninformed-search)
    1. [Breadth-First Search (BFS)](#21-breadth-first-search-bfs)
    2. [Depth-First Search (DFS)](#22-depth-first-search-dfs)
3. [Informed Search](#3-informed-search)
    1. [Uniform Cost Search (UCS)](#21-uniform-cost-search-ucs)

---

### 1) Preliminary Steps

Importing the classes needed for the algorithm

In [2]:
from __future__ import annotations
from typing import Protocol, Iterator, Tuple, TypeVar, Optional
import collections

T = TypeVar('T')
Location = TypeVar('Location')

Defining the `Graph` and the `SimpleGraph` classes, which are needed to define the problem

In [3]:
class Graph(Protocol):
    def neighbors(self, id: Location) -> list[Location]:
        pass

class SimpleGraph:
    def __init__(self):
        self.edges: dict[Location, list[Location]] = {}

    def neighbors(self, id: Location) -> list[Location]:
        return self.edges[id]

Defining some utility functions (so `from_id_width()`, `draw_tile()` and `draw_grid()`) for dealing with square grids

In [4]:
def from_id_width(id, width):
    return (id % width, id // width)

def draw_tile(graph, id, style):
    r = " × "
    if id in graph.swamps: r = "░░░"
    if 'number' in style and id in style['number']: r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " → "
        if x2 == x1 - 1: r = " ← "
        if y2 == y1 + 1: r = " ↓ "
        if y2 == y1 - 1: r = " ↑ "
    if 'path' in style and id in style['path']:   r = " ◆ "
    if 'start' in style and id == style['start']: r = "[S]"
    if 'goal' in style and id == style['goal']:   r = "[G]"
    if id in graph.walls: r = "███"
    return r

def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph, (x, y), style), end="")
        print()
    print("---" * graph.width)

Defining the `SquareGrid` class and the `reconstruct_path()` function, which reconstructs visually the path discovered by the BFS algorithm

In [5]:
GridLocation = Tuple[int, int]

class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
        self.swamps: list[GridLocation] = []

    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height

    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls

    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

def reconstruct_path(came_from: dict[Location, Location],
                     start: Location, goal: Location) -> list[Location]:
    current: Location = goal
    path: list[Location] = []
    if goal not in came_from: # no path was found
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

Finally, build the first diagram, and provide the `start` and `goal` coordinates

In [6]:
DIAGRAM1_WALLS = [from_id_width(id, width=30) for id in [21,22,51,52,81,82,93,94,111,112,123,
                                                        124,133,134,141,142,153,154,163,164,
                                                        171,172,173,174,175,183,184,193,194,
                                                        201,202,203,204,205,213,214,223,224,
                                                        243,244,253,254,273,274,283,284,303,
                                                        304,313,314,333,334,343,344,373,374,
                                                        403,404,433,434]]
g = SquareGrid(30, 15)
g.walls = DIAGRAM1_WALLS

start = (8, 7)
goal = (18, 8)

draw_grid(g, start=start, goal=goal)

__________________________________________________________________________________________
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  × ███████████████ ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  × ███████████████ ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  × [S] ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  × [G] ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 

---

### 2) Uninformed Search

Before building the algorithms, two last classes must be defined, and these are the `FIFO` and `LIFO` classes, which will be the basis of our frontiers in the BFS and DFS algorithms respectively:

In [7]:
class FIFO:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: collections.deque[T] = collections.deque()

    def empty(self) -> bool:
        """Checks whether a FIFO queue is empty or not

        Returns:
                status (bool):          if `true`, then the queue is empty; if `false`,
                                        then the queue still contains some elements """
        return not self.elements
    
    def put(self, x: T) -> None:
        """Puts into the FIFO queue an item `x`

        Parameters:
                x (T):                  the item to put inside the queue """
        self.elements.append(x)

    def get(self) -> T:
        """Pops the left-most item from the queue, and returns it

        Returns:
                x (T):                  the item popped from the queue """
        return self.elements.popleft()

In [8]:
class LIFO:
    """A simple queue implementation using `collections.deque`."""
    def __init__(self) -> None:
        self.elements: collections.deque[T] = collections.deque()

    def empty(self) -> bool:
        """Checks whether a FIFO queue is empty or not

        Returns:
                status (bool):          if `true`, then the queue is empty; if `false`,
                                        then the queue still contains some elements """
        return not self.elements

    def put(self, x: T) -> None:
        """Puts into the FIFO queue an item `x`

        Parameters:
                x (T):                  the item to put inside the queue """
        self.elements.append(x)

    def get(self) -> T:
        """Pops the first item from the queue, and returns it

        Returns:
                x (T):                  the item popped from the queue """
        return self.elements.pop()

#### 2.1) Breadth First Search (BFS)

Now we can finally build the `breadth_first_search()` algorithm

In [9]:
def breadth_first_search(graph: Graph, start: Location, goal: Location):
    """Implementation of a BFS algorithm in Python
    
    Parameters:
                graph (`Graph`): a graph of the problem
                start (`Location`): the point where the algorithm should start
                goal (`Location`): the goal of the problem
                
    Returns
                solution (`dict[Location, Location | None]`): if solution returns `None` then
                                                                the algorithm failed, otherwise
                                                                it will return the solution"""
    frontier = FIFO()       # Initialize the frontier
    frontier.put(start)     # Put the start in the frontier
    came_from: dict[Location, Optional[Location]] = {}  # Create a dictionary
                                                        # that returns from where
                                                        # we reached a node
    came_from[start] = None     # Naturally, we don't come from anywhere
                                # if we consider the start

    while frontier:     # While the frontier is not empty
        current: Location = frontier.get()      # Store into "current" a Location, 
                                                # which is the first item popped
                                                # from the frontier
        if current == goal:     # If the currently examined node is the goal...
            break       # ...exit from the loop and return it
        for next in graph.neighbors(current):   # For all the neighbours of "current"
            if next not in came_from:       # If we didn't examine the neighbour...
                frontier.put(next)      # ...then put it in the frontier...
                came_from[next] = current   # ...and store it in "came_from"
    
    return came_from    # Return the node


We can finally see the result of the algorithm: the arrows denote all the cells where the algorithm passed, and they denote the path from the goal to the start. In the next output, the path found by the algorithm is denoted by the `◆` symbols

In [10]:
parents = breadth_first_search(g, start, goal)
draw_grid(g, point_to=parents, start=start, goal=goal)

__________________________________________________________________________________________
 →  →  →  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ←  ←  ←  ←  ← ██████ ×  ×  ×  ×  ×  ×  × 
 →  →  →  →  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ←  ←  ←  ←  ←  ← ██████ ×  ×  ×  ×  ×  ×  × 
 →  →  →  →  →  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ←  ←  ←  ←  ←  ←  ← ██████ ×  ×  ×  ×  ×  ×  × 
 →  →  ↑ ██████ ↓  ↓  ↓  ↓  ↓  ↓  ↓  ↓  ←  ←  ←  ←  ←  ←  ←  ← ██████ ×  ×  ×  ×  ×  ×  × 
 →  ↑  ↑ ██████ ↓  ↓  ↓  ↓  ↓  ↓  ↓  ← ██████ ↑  ←  ←  ←  ←  ← ██████ ×  ×  ×  ×  ×  ×  × 
 ↑  ↑  ↑ ██████ →  ↓  ↓  ↓  ↓  ↓  ←  ← ██████ ↑  ↑  ←  ←  ←  ← ███████████████ ×  ×  ×  × 
 ↑  ↑  ↑ ██████ →  →  ↓  ↓  ↓  ←  ←  ← ██████ ↑  ↑  ↑  ←  ←  ← ███████████████ ×  ×  ×  × 
 ↓  ↓  ↓ ██████ →  →  → [S] ←  ←  ←  ← ██████ ↑  ↑  ↑  ↑  ←  ←  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ↓  ↓  ↓ ██████ →  →  ↑  ↑  ↑  ←  ←  ← ██████ ↑  ↑  ↑ [G] ↑  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ↓  ↓  ↓ ██████ →  ↑  ↑  ↑  ↑  ↑  ←  ← ██████ ↑  ↑  ↑  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 

In [11]:
draw_grid(g, path=reconstruct_path(parents, start=start, goal=goal), start=start, goal=goal)

__________________________________________________________________________________________
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  ◆  ◆  ◆  ◆  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ◆  ◆ ██████ ◆  ◆  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ◆  ◆  × ██████ ×  ◆  ◆  ×  ×  × ███████████████ ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ◆  ◆  ×  × ██████ ×  ×  ◆  ◆  ×  × ███████████████ ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  × [S] ◆  ×  ×  × ██████ ×  ×  ×  ◆  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  × [G] ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  × ██████ ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 

#### 2.2) Depth-First Search (DFS)

The Depth-First Search algorithm explores one branch at a time, and is implemented with the `LIFO` queue:

In [18]:
def depth_first_search(graph: Graph, start: Location, goal: Location):
    """Implementation of a DFS algorithm in Python
    
    Parameters:
                graph (`Graph`): a graph of the problem
                start (`Location`): the point where the algorithm should start
                goal (`Location`): the goal of the problem
                
    Returns
                solution (`dict[Location, Location | None]`): if solution returns `None` then
                                                                the algorithm failed, otherwise
                                                                it will return the solution"""
    frontier = LIFO()   # Initialize the frontier
    frontier.put(start) # Put the start into the frontier
    came_from: dict[Location, Optional[Location]] = {}  # Save into the dictionary from where
                                                        # we came
    came_from[start] = None # Naturally, we don't come from anywhere with the start

    while not frontier.empty(): # While the frontier is not empty
        current: Location = frontier.get()  # Get the first element of the frontier
        if current == goal: # If the currently examinated element is the goal...
            break   # ...then exit from the loop
        for next in graph.neighbors(current):   # For each neighbour of the examinated node...
            if next not in came_from:   # ...if we didn't examine the node yet...
                frontier.put(next)  # ...put the node into the frontier...
                came_from[next] = current   # ...and set that we reached such node from the previous element

    return came_from    # Finally, return all the paths

This is the result of the algorithm:

In [19]:
parents = depth_first_search(g, start, goal)
draw_grid(g, point_to=parents, start=start, goal=goal)

_________________________________
 →  →  →  →  ↓  ↓  ←  ←  ←  →  ↓ 
 →  ↑  ↑  →  →  ↓  ↓  ↑  ←  ←  ↓ 
 ↑  ↑  ←  ←  →  →  ↓  ↓  ↑  ←  ← 
 ↑  ↓  ↑  ↑  ←  →  →  ↓ [G] ↑  ← 
 → [S] ←  ↑  ↑  ←  →  →  ↓  ↓  ↑ 
 →  ↑  ←  ←  ↓  ↓  ↑  →  →  ↓  ↓ 
 ×  ↑  ↑  ←  ←  ←  ←  ×  →  →  ↓ 
 × █████████ ↑  ↑  ←  ←  ×  →  ↓ 
 × █████████░░░ ↑  ↑  ←  ←  ↓  ↓ 
 ×  ×  ×  ×  ×  ×  ↑  ↑  ←  ←  ← 
---------------------------------


In [20]:
draw_grid(g, path=reconstruct_path(parents, start=start, goal=goal),start=start, goal=goal)

_________________________________
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ×  ×  ×  × ░░░░░░ ×  ×  ×  ×  × 
 ×  ×  ×  × ░░░░░░░░░ ×  ×  ×  × 
 ×  ×  ×  × ░░░░░░░░░░░░[G] ×  × 
 × [S] × ░░░░░░░░░░░░░░░ ◆  ×  × 
 ×  ◆  ◆ ░░░░░░░░░░░░░░░ ◆  ◆  × 
 ×  ×  ◆  ◆  ◆  ◆ ░░░ ×  ×  ◆  ◆ 
 × █████████░░░ ◆  ◆  ×  ×  ×  ◆ 
 × █████████░░░░░░ ◆  ◆  ×  ×  ◆ 
 ×  ×  ×  ×  ×  ×  ×  ◆  ◆  ◆  ◆ 
---------------------------------


---

### 3) Informed Search

In order to make an informed search algorithm, we first want to complicate the things a bit. We now introduce also a cost into the grid, which is given by the "swamp": crossing the swamp has an higher cost than the normal path.

In [12]:
from os import get_blocking
class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location) -> float: pass

class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: dict[GridLocation, float] = {}

    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        if to_node in self.swamps:
            return self.weights.get(to_node, 5)
        return self.weights.get(to_node, 1)


D4_walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
D4_swamp = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}


start, goal = (1, 4), (8, 3)
g = GridWithWeights(11, 10)
g.walls = D4_walls
g.swamps = D4_swamp
draw_grid(g, start=start, goal=goal)

_________________________________
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
 ×  ×  ×  × ░░░░░░ ×  ×  ×  ×  × 
 ×  ×  ×  × ░░░░░░░░░ ×  ×  ×  × 
 ×  ×  ×  × ░░░░░░░░░░░░[G] ×  × 
 × [S] × ░░░░░░░░░░░░░░░ ×  ×  × 
 ×  ×  × ░░░░░░░░░░░░░░░ ×  ×  × 
 ×  ×  ×  × ░░░░░░░░░ ×  ×  ×  × 
 × █████████░░░░░░░░░ ×  ×  ×  × 
 × █████████░░░░░░ ×  ×  ×  ×  × 
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
---------------------------------


In order to implement our heuristic function, we first need to create a priority queue, since we'll need to order the nodes in the frontier depending on the heuristic:

In [13]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, T]] = []

    def empty(self) -> bool:
        return not self.elements

    def put(self, item: T, priority: float):
        heapq.heappush(self.elements, (priority, item))

    def get(self) -> T:
        return heapq.heappop(self.elements)[1]

#### 2.1) Uniform Cost Search (UCS)

Now we can design the first algorithm, which will be a Uniform Cost Search (UCS) algorithm:

In [14]:
def uc_search(graph: WeightedGraph, start: Location, goal: Location):
    """Implementation of a Uniform Cost Search algorithm in Python
    
    Parameters:
                graph (`WeightedGraph`): a weighted graph of the problem
                start (`Location`): the point where the algorithm should start
                goal (`Location`): the goal of the problem
                
    Returns
                solution (`tuple[dict[Location, Location | None], dict[Location, float]]`): if solution returns `None` then
                                                                                            the algorithm failed, otherwise
                                                                                            it will return the solution. A second
                                                                                            dictionary is returned, which
                                                                                            defines the cost needed to reach
                                                                                            each location"""
    frontier = PriorityQueue()  # Initialize the frontier as a priority queue
    frontier.put(start, 0)      # Put the start into the queue
    
    came_from: dict[Location, Optional[Location]] = {}      # Store the path and from
                                                            # where we came
    came_from[start] = None     # Since the start is the first node, set that
                                # we didn't come from anywhere

    cost_so_far: dict[Location, float] = {}     # Store the cost up to a node n
    cost_so_far[start] = 0      # The cost to reach the start from the start is 0

    while not frontier.empty():     # While the frontier is not empty
        current: Location = frontier.get()  # Get the first element of the queue
        if current == goal:     # If such element is the goal...
            break       # ...exit from the loop
        for next in graph.neighbors(current):   # For each neighbour of the element
            new_cost = cost_so_far[current] + graph.cost(current, next) # Get the new cost
                                                                        # needed to reach the
                                                                        # neighbour
            if (next not in cost_so_far) or (new_cost < cost_so_far[next]): # If we didn't explore
                                                                            # such node before...
                cost_so_far[next] = new_cost    # ...store the cost up to the node...
                frontier.put(next, new_cost) # ...put in the queue the node with its cost so far,
                                             # which now becomes the priority...
                came_from[next] = current   # ...and store the path to the node

    return came_from, cost_so_far   # Finally, return the path and the costs

We can now preview the results of the algorithm: first, we check all the explored cells of the algorithm

In [15]:
parents, cost = uc_search(g, start, goal)
draw_grid(g, point_to=parents, start=start, goal=goal)

_________________________________
 ↓  ↓  ←  ←  ←  ←  ←  ←  ←  ←  ← 
 ↓  ↓  ←  ←  ←  ↑  ↑  ←  ←  ←  ← 
 ↓  ↓  ←  ←  ←  ←  ↑  ↑  ←  ←  × 
 ↓  ↓  ←  ←  ←  ←  ←  ↑ [G] ×  × 
 → [S] ←  ←  ←  ← ░░░░░░ ×  ×  × 
 ↑  ↑  ←  ←  ←  ← ░░░░░░ ×  ×  × 
 ↑  ↑  ←  ←  ←  ←  ←  ×  ×  ×  × 
 ↑ █████████ ↑  ←  ↓  ↓  ×  ×  × 
 ↑ █████████ ↓  ↓  ↓  ←  ←  ×  × 
 ↑  ←  ←  ←  ←  ←  ←  ←  ←  ×  × 
---------------------------------


The final path of the algorithm is, as usual, the one denoted by the `◆` symbol:

In [16]:
draw_grid(g, path=reconstruct_path(parents, start=start, goal=goal),start=start, goal=goal)

_________________________________
 ×  ◆  ◆  ◆  ◆  ◆  ◆  ×  ×  ×  × 
 ×  ◆  ×  × ░░░░░░ ◆  ◆  ×  ×  × 
 ×  ◆  ×  × ░░░░░░░░░ ◆  ◆  ×  × 
 ×  ◆  ×  × ░░░░░░░░░░░░[G] ×  × 
 × [S] × ░░░░░░░░░░░░░░░ ×  ×  × 
 ×  ×  × ░░░░░░░░░░░░░░░ ×  ×  × 
 ×  ×  ×  × ░░░░░░░░░ ×  ×  ×  × 
 × █████████░░░░░░░░░ ×  ×  ×  × 
 × █████████░░░░░░ ×  ×  ×  ×  × 
 ×  ×  ×  ×  ×  ×  ×  ×  ×  ×  × 
---------------------------------


Finally, we can see the cost of all the cells explored by the algorithm:

In [17]:
draw_grid(g, number=cost, start=start, goal=goal)

_________________________________
 5  4  5  6  7  8  9  10 11 12 13
 4  3  4  5  10 13 10 11 12 13 14
 3  2  3  4  9  14 15 12 13 14 × 
 2  1  2  3  8  13 18 17[G] ×  × 
 1 [S] 1  6  11 16░░░░░░ ×  ×  × 
 2  1  2  7  12 17░░░░░░ ×  ×  × 
 3  2  3  4  9  14 19 ×  ×  ×  × 
 4 █████████ 14 19 18 15 ×  ×  × 
 5 █████████ 15 16 13 14 15 ×  × 
 6  7  8  9  10 11 12 13 14 ×  × 
---------------------------------
